# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
# from d2l import torch as d2l
# from torch import nn
from sklearn.model_selection import train_test_split
import csv
# import torch

In [2]:
import os

# Check if running in a conda environment
if 'CONDA_DEFAULT_ENV' in os.environ:
    print("Running in a conda environment. Conda environment:", os.environ['CONDA_DEFAULT_ENV'])
else:
    print("Not running in a conda environment.")

# Check if running in a virtual environment
if 'VIRTUAL_ENV' in os.environ:
    print("Running in a virtual environment. Virtual environment:", os.environ['VIRTUAL_ENV'])
else:
    print("Not running in a virtual environment.")

Running in a conda environment. Conda environment: ML_Labs
Not running in a virtual environment.


# Load Dataset

In [3]:
kaggle_data = pd.read_json('train.json')
kaggle_data.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [4]:
# Split the first 90% of the data as the train set
test_size = round(len(kaggle_data)*0.1) 
train_size = len(kaggle_data) - test_size

# train_set, test_set = train_test_split(kaggle_data, test_size=len(kaggle_data)-train_size, random_state = False)
train_set = kaggle_data.iloc[:train_size].copy()
test_set = kaggle_data.iloc[train_size:].copy()

# train_set
test_size

kaggle_data['full_text']

0       Design Thinking for innovation reflexion-Avril...
1       Diego Estrada\n\nDesign Thinking Assignment\n\...
2       Reporting process\n\nby Gilberto Gamboa\n\nCha...
3       Design Thinking for Innovation\n\nSindy Samaca...
4       Assignment:  Visualization Reflection  Submitt...
                              ...                        
6802    EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE    My w...
6803    Why Mind Mapping?\n\nMind maps are graphical r...
6804    Challenge\n\nSo, a few months back, I had chos...
6805    Brainstorming\n\nChallenge & Selection\n\nBrai...
6806    Mind Mapping\n\nChallenge\n\nMy consulting tea...
Name: full_text, Length: 6807, dtype: object

In [5]:
essays = kaggle_data['full_text'].tolist()
tokens = kaggle_data['tokens'].tolist()

# Word Embedding

In [6]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [7]:
# import spaCy's language model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [8]:
test_set['lemmatized_text'] = lemmatization(test_set['full_text'])

In [9]:
train_set['lemmatized_text'] = lemmatization(train_set['full_text'])

In [10]:
train_set.to_csv("lem_train_set.csv", index=False)

In [11]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [46]:
# Convert the input text into a tensor
input_text = tf.constant(['Thanks'])


sequence_len = tf.constant([len(tokens) for tokens in kaggle_data['tokens']])


# Pass the tensor to the ELMo model
embeddings = elmo.signatures['tokens'](sequence_len=sequence_len, tokens=input_text)

InvalidArgumentError: Graph execution error:

Detected at node strided_slice_1 defined at (most recent call last):
  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\runpy.py", line 86, in _run_code

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\Users\sblan\miniconda3\envs\ML_Labs\lib\asyncio\base_events.py", line 603, in run_forever

  File "C:\Users\sblan\minicond

In [23]:
embeddings

{'lstm_outputs1': <tf.Tensor: shape=(1, 1, 1024), dtype=float32, numpy=
 array([[[-0.20643494, -0.4799158 ,  0.08882089, ...,  0.04712017,
          -0.14548695,  0.18695112]]], dtype=float32)>,
 'lstm_outputs2': <tf.Tensor: shape=(1, 1, 1024), dtype=float32, numpy=
 array([[[-0.26701212, -1.4938462 , -0.25769508, ...,  0.23783995,
          -0.04308726,  0.42713493]]], dtype=float32)>,
 'word_emb': <tf.Tensor: shape=(1, 1, 512), dtype=float32, numpy=
 array([[[ 0.47496802, -0.33135816, -0.4002301 ,  0.43449467,
           0.25025582, -0.2817593 , -0.60255706,  0.18856972,
           0.9883196 , -0.47195002,  1.0381509 , -0.9946794 ,
           0.08266845,  0.52927065,  0.02863112, -0.61167485,
          -0.18608254,  0.6679682 ,  0.9596645 , -0.16537662,
          -0.9713387 , -1.3922101 , -1.5799298 ,  1.8138862 ,
           0.05286388,  0.78104705, -0.68480945,  0.6325536 ,
          -0.73391354, -0.406277  , -0.41568005, -0.13266781,
           0.5182327 ,  0.55705553, -0.75603443,

In [34]:
# Find longest len of tokens
max_index = kaggle_data['tokens'].apply(len).idxmax()
max_length = len(kaggle_data.loc[max_index, 'tokens'])

# Get the row with the longest list of tokens
row_with_max_tokens = kaggle_data.loc[max_index]

print("Row with the longest list of tokens:")
# print(row_with_max_tokens)
print("Length of the longest list of tokens:", max_length)



Row with the longest list of tokens:
Length of the longest list of tokens: 3298


In [14]:
# Save the model
# Define the path where you want to save the model
saved_model_path = "./saved_model"

# Save the model
tf.saved_model.save(elmo, saved_model_path)

INFO:tensorflow:Assets written to: ./saved_model\assets


INFO:tensorflow:Assets written to: ./saved_model\assets


In [15]:
elmo.Trainable = True

restore: <function _EagerSavedModelLoader.load.<locals>.<lambda> at 0x000002303F95F6D0>
_self_setattr_tracking: True
_self_unconditional_checkpoint_dependencies: [TrackableReference(name=initializer, ref=<tensorflow.python.saved_model.load_v1_in_v2._Initializer object at 0x000002307942B5E0>), TrackableReference(name=asset_paths, ref=ListWrapper([])), TrackableReference(name=signatures, ref=_SignatureMap({'tokens': <ConcreteFunction () -> Dict[['lstm_outputs1', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['lstm_outputs2', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['word_emb', TensorSpec(shape=(None, None, 512), dtype=tf.float32, name=None)], ['elmo', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['sequence_len', TensorSpec(shape=(None,), dtype=tf.int32, name=None)], ['default', TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None)]] at 0x2307942B3D0>, 'default': <ConcreteFunction () -> Dict[['lstm_outputs1', T

# Embed the whole dataset